#Predition EUI using regression algorithms. First of all, load the dataset.

In [1]:
from aibpd.data.database import Database
from aibpd.data.preprocessing import PreprocessingCBECS
import numpy as np
import pandas as pd
db=Database()
CBECS_DF=db.select()

Please select a database to continue the analysis
Dataset in C:/Users/tzcha/Documents/aibpd/resources/CBECS2012.csv has been loaded!


In [4]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(CBECS_DF)

ModuleNotFoundError: No module named 'sklearn.impute'

In [19]:
#preprocessing for regression
prep4eui=PreprocessingCBECS()
CBECS_DF=prep4eui.prep4EUIReg(CBECS_DF)

In [20]:
CBECS_DF=CBECS_DF[CBECS_DF['principleActivity']>=2][CBECS_DF['principleActivity']<3]
CBECS_DF=CBECS_DF.dropna()
CBECS_DF['MONUSEC']=pd.cut(CBECS_DF['MONUSE'],[0,6,9,10,13],labels=[1,2,3,4])
CBECS_DF['OCCUPYPC']=pd.cut(CBECS_DF['OCCUPYP'],5,labels=range(5))
CBECS_DF['HEATPC']=pd.cut(CBECS_DF['HEATP'],[-10,50,90,101],labels=[1,2,3])
CBECS_DF['MONUSEC']=CBECS_DF['MONUSEC'].astype('int')
CBECS_DF['OCCUPYPC']=CBECS_DF['OCCUPYPC'].astype('int')
CBECS_DF['HEATPC']=CBECS_DF['HEATPC'].astype('int')
CBECS_DF['EUI']=CBECS_DF['EUI'].astype('float')

In [21]:
db.datasetDF=CBECS_DF

In [23]:
db.save('CBECS4Reg')

datasetDF has been saved to C:/Users/tzcha/Documents/aibpd/resources/CBECS4Reg.csv


In [10]:
featureList=['buildingAreaCategory','buildingShape','censusRegion','climateZone','HDD65',\
            'HEATPC','HVACUpgrade','insulationUpgrade','MAINCL','MAINHT','MONUSEC','numEmployeesCategory',\
             'numFloors','OCCUPYP','OPNWE',\
            'OWNTYPE', 'region','RENWLL','lightingUpgrade','roofConstuction','STUSED',\
            'OPEN24','wallConstruction',\
            'WHOPPR','WINTYP','WKHRSC','WWR','yrConstructionCategory','EUI']
corrMattDF=CBECS_DF[featureList]
column=featureList

In [24]:
x=input('input a value')

input a value3


In [18]:
boroughs = list(boroughs[boroughs.values > 100].index)
boroughs

[2.0, 0.0, 1.0]

In [6]:
db.save(name='CBECS4Reg')

datasetDF has been saved to C:/Users/tzcha/Documents/aibpd/resources/CBECS4Reg.csv


In [34]:
#feature selection
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
#for j in range(1,25):
    #print('Training dataset shape:', X_train.shape, y_train.shape)
    #print('Testing dataset shape:', X_test.shape, y_test.shape)
    #j1=25-j
clf = svm.SVR(gamma=0.001, C=1.0, epsilon=0.2)
sfs1 = sfs(clf,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='r2',
           cv=5)

sfs1 = sfs1.fit(corrMattDF.values[:,:-1], corrMattDF.values[:,-1])
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    5.4s finished

[2019-05-09 16:48:06] Features: 1/5 -- score: -0.04721480608460511[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    5.4s finished

[2019-05-09 16:48:12] Features: 2/5 -- score: -0.04364721025698781[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    5.3s finished

[2019-05-09 16:48:17] Features: 3/5 -- score: -0.04288109806714937[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    5.1s finished

[2019-05-09 16:48:23] Features: 4/5 -- score: -0.04259167047636212[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[5, 11, 15, 21, 25]


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    5.0s finished

[2019-05-09 16:48:28] Features: 5/5 -- score: -0.042394580225644685

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    corrMattDF.values[:,:-1],
    corrMattDF.values[:,-1],
    test_size=0.2,
    random_state=42)
y_train = y_train.ravel()
y_test = y_test.ravel()

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
errorList=[]
X1=corrMattDF.values[:,[4,6,12,16,26]]
Y1=corrMattDF.values[:,-1]
for train_index, test_index in kf.split(X1,Y1):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = Y1[train_index], Y1[test_index]
    clf2 = svm.SVR(gamma=0.001, C=1.0, epsilon=0.2)
    clf2.fit(X_train,y_train) 
    error=1-clf2.score(X_test,y_test)
    errorList.append(error)
error2=np.mean(errorList)
print(1-error2)

-0.04228885275308447


In [11]:
m,n=corrMattDF.shape

In [27]:
m_train=int(0.8*m)
clf3 = svm.SVR(gamma=0.001, C=1.0, epsilon=0.2)
clf3.fit(corrMattDF.values[:m_train,:-1],corrMattDF.values[:m_train,-1])

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma=0.001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [28]:
for i in range(m_train,m):
    print(clf3.predict(corrMattDF.values[i,:-1].reshape(1,29)),'y',corrMattDF.values[i,-1])

[59.50190628] y 39.78
[58.18522034] y 59.840812
[62.74369042] y 123.93555294117647
[58.99929213] y 42.46987
[57.29161129] y 124.66727272727273
[63.64901043] y 44.16577777777778
[59.54787279] y 30.802976
[58.42282967] y 48.28807692307692
[60.35459634] y 43.65266666666667
[61.71456779] y 99.02422975770243
[58.83373505] y 67.55933333333333
[57.39451672] y 42.228125
[61.30105259] y 104.00471571428571
[59.11199935] y 137.202
[61.04054519] y 51.8625
[59.73590764] y 124.67267567567568
[58.5537326] y 104.86287878787878
[60.8001074] y 54.69639344262295
[64.43516183] y 143.07925333333333
[61.49107998] y 19.2244375
[61.42856613] y 34.99127272727273
[55.48043353] y 63.55672
[60.09477606] y 37.934416666666664
[60.21007188] y 446.69026
[56.24560135] y 82.55198666666666
[59.2741486] y 93.74834166666666
[59.20329506] y 46.80758
[58.84537224] y 79.89874929577465
[58.91304263] y 107.44053142857143
[60.43673808] y 135.2490909090909
[61.48172075] y 42.231764705882355
[58.84758563] y 98.5862
[60.70573607] 